In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
path = '../Data/Raw/재현데이터.xlsx'

df_origin = pd.read_excel(path)

In [3]:
df_origin.head(1)

,일련번호,기업명,select,사업자등록번호,설립연도,기업구분,22년매출,임직원수,업종,구분,...,B_05,B_06,B_07,B_08,B_09,B_10,B_11,B_12,B_14,B_15
0,A02,에이닷(주),미선정,187-88-02389,2021,중소기업,477.0,7.0,서비스,수요기업,...,4,5,5,4,5,5,6,6,6,6


In [4]:
changeDict = {1 : 0, 
              2 : 16.67 * 1, 
              3 : 16.67 * 2, 
              4 : 16.67 * 3, 
              5 : 16.67 * 4, 
              6 : 16.67 * 5, 
              7 : 100}

for column in df_origin.columns[10:]:
    df_origin.loc[:, column] = df_origin.loc[:, column].apply(lambda x : changeDict[x]).values

In [5]:
df_origin.head(1)

,일련번호,기업명,select,사업자등록번호,설립연도,기업구분,22년매출,임직원수,업종,구분,...,B_05,B_06,B_07,B_08,B_09,B_10,B_11,B_12,B_14,B_15
0,A02,에이닷(주),미선정,187-88-02389,2021,중소기업,477.0,7.0,서비스,수요기업,...,50.01,66.68,66.68,50.01,66.68,66.68,83.35,83.35,83.35,83.35


In [6]:
from sdv.metadata import SingleTableMetadata

In [7]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data = df_origin)
python_dict = metadata.to_dict()

In [8]:
python_dict

{'columns': {'일련번호': {'sdtype': 'id'},
  '기업명': {'sdtype': 'unknown', 'pii': True},
  'select': {'sdtype': 'categorical'},
  '사업자등록번호': {'sdtype': 'unknown', 'pii': True},
  '설립연도': {'sdtype': 'numerical'},
  '기업구분': {'sdtype': 'categorical'},
  '22년매출': {'sdtype': 'numerical'},
  '임직원수': {'sdtype': 'numerical'},
  '업종': {'sdtype': 'categorical'},
  '구분': {'sdtype': 'categorical'},
  'A_01': {'sdtype': 'numerical'},
  'A_02': {'sdtype': 'numerical'},
  'A_03': {'sdtype': 'numerical'},
  'A_04': {'sdtype': 'numerical'},
  'A_05': {'sdtype': 'numerical'},
  'A_06': {'sdtype': 'numerical'},
  'A_07': {'sdtype': 'numerical'},
  'A_08': {'sdtype': 'numerical'},
  'A_09': {'sdtype': 'numerical'},
  'A_10': {'sdtype': 'numerical'},
  'A_11': {'sdtype': 'numerical'},
  'A_12': {'sdtype': 'numerical'},
  'A_13': {'sdtype': 'numerical'},
  'A_14': {'sdtype': 'numerical'},
  'A_15': {'sdtype': 'numerical'},
  'A_16': {'sdtype': 'numerical'},
  'A_17': {'sdtype': 'numerical'},
  'A_18': {'sdtype':

In [9]:
from sdv.single_table import CTGANSynthesizer

synthesizer = CTGANSynthesizer(
    metadata,
    enforce_rounding=False,
    epochs=100,
    verbose=True
)

# synthesizer.fit(df_origin)

In [10]:
synthesizer.fit(df_origin)

Gen. (-1.56) | Discrim. (0.46): 100%|██████████| 100/100 [01:40<00:00,  1.01s/it]


In [11]:
synthetic_data =synthesizer.sample(
    num_rows=200,
    batch_size=10
)

Sampling rows: 100%|██████████| 200/200 [00:08<00:00, 22.71it/s]


In [12]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data=df_origin,
    synthetic_data=synthetic_data,
    metadata=metadata
)

Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 1596/1596 [00:10<00:00, 149.59it/s]

Overall Quality Score: 69.69%

Properties:
- Column Shapes: 62.98%
- Column Pair Trends: 76.4%


In [13]:
from sdv.evaluation.single_table import run_diagnostic

diagnostic_report = run_diagnostic(
    real_data=df_origin,
    synthetic_data=synthetic_data,
    metadata=metadata)

Generating report ...
(3/3) Evaluating Synthesis: : 100%|██████████| 1/1 [00:12<00:00, 12.55s/it]

Diagnostic Results:

SUCCESS:
✓ The synthetic data covers over 90% of the categories present in the real data
✓ The synthetic data covers over 90% of the numerical ranges present in the real data
✓ The synthetic data follows over 90% of the min/max boundaries set by the real data
✓ Over 90% of the synthetic rows are not copies of the real data


In [14]:
from sdv.evaluation.single_table import get_column_plot

fig = get_column_plot(
    real_data=df_origin,
    synthetic_data=synthetic_data,
    column_name='B_05',
    metadata=metadata
)
    
fig.show()

In [17]:
fig = get_column_plot(
    real_data=df_origin,
    synthetic_data=synthetic_data,
    column_name='B_05',
    metadata=metadata
)
    
fig.show()

In [16]:
fig = get_column_plot(
    real_data=df_origin,
    synthetic_data=synthetic_data,
    column_name='A_03',
    metadata=metadata
)
    
fig.show()

In [52]:
synthetic_data.head(2)

,일련번호,기업명,select,사업자등록번호,설립연도,기업구분,22년매출,임직원수,업종,구분,...,B_05,B_06,B_07,B_08,B_09,B_10,B_11,B_12,B_14,B_15
0,sdv-id-20,sdv-pii-pjb0i,미선정,sdv-pii-a4fy5,2016,중소기업,0.0,22.467499,기타,수요기업,...,50.618442,61.256249,74.011654,100.0,46.472961,36.566602,100.000000,82.337816,52.594663,87.333231
1,sdv-id-21,sdv-pii-g0atv,미선정,sdv-pii-le3wo,2016,중소기업,0.0,15.156290,제조,공급기업,...,66.824115,36.494692,52.074491,100.0,97.265958,20.227791,80.103044,24.279952,100.000000,33.340000


In [53]:
# synthetic_data.to_csv('../Data/Preprocessed/synthetic_data.csv', index = False, encoding = 'utf-8-sig')

In [55]:
def changeReverseDict(x, changeDict):
    if x < changeDict[2]:
        return 1
    elif (changeDict[2] <= x) & (x < changeDict[3]):
        return 2
    elif (changeDict[3] <= x) & (x < changeDict[4]):
        return 3
    elif (changeDict[4] <= x) & (x < changeDict[5]):
        return 4
    elif (changeDict[5] <= x) & (x < changeDict[6]):
        return 5
    elif (changeDict[6] <= x) & (x < changeDict[7]):
        return 6
    else:
        return 7

In [59]:
synthetic_data_copy = synthetic_data.copy()

for column in synthetic_data_copy.columns[10:]:
    synthetic_data_copy.loc[:, column] = synthetic_data_copy.loc[:, column].apply(lambda x : changeReverseDict(x, changeDict)).values

# synthetic_data_copy.to_csv('../Data/Preprocessed/synthetic_data_category.csv', index = False, encoding = 'cp949')